> # **Imports**

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import tensorflow as tf
import tensorflow_probability as tfp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
import pickle
import tensorflow_hub as hub

2024-07-16 13:45:56.377404: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-16 13:45:56.377518: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-16 13:45:56.500454: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


> # **Data processing**

In [2]:
embed = hub.load("https://kaggle.com/models/google/universal-sentence-encoder/TensorFlow2/universal-sentence-encoder/1")
db = pd.read_excel('/kaggle/input/tnveds/TNVED.xlsx')

Attaching 'None' from model 'google/universal-sentence-encoder/tensorflow2/universal-sentence-encoder/1' to your Kaggle notebook...


In [9]:
y_train = tf.constant(db['Number'])
X_train_embeddings = embed(db["Name"].values)
X_train_matrix = X_train_embeddings['outputs'].numpy()
X_train_matrix.shape

(13927, 512)

> # **Class**

In [11]:
class TFNaiveBayesClassifier:
    dist = None
    print(2)

    # X is the matrix containing the vectors for each sentence
    # y is the list target values in the same order as the X matrix
    def fit(self, X, y):
        print(3)
        unique_y = np.unique(y)  # unique target values: 0,1
        print(unique_y)
        # `points_by_class` is a numpy array the size of
        # the number of unique targets.
        # in each item of the list is another list that contains the vector
        # of each sentence from the same target value
        points_by_class = np.asarray([np.asarray(
            [np.asarray(
                X.iloc[x, :]) for x in range(0, len(y)) if y[x] == c]) for c in unique_y])
        print(13)
        mean_list = []
        var_list = []
        for i in range(0, len(points_by_class)):
            mean_var, var_var = tf.nn.moments(tf.constant(points_by_class[i]), axes=[0])
            mean_list.append(mean_var)
            var_list.append(var_var)
            print(0)
        mean = tf.stack(mean_list, 0)
        var = tf.stack(var_list, 0)
        # Create a 3x2 univariate normal distribution with the
        # known mean and variance
        self.dist = tfp.distributions.Normal(loc=mean, scale=tf.sqrt(var))
        print(4)

    def predict(self, X):
        print(5)
        assert self.dist is not None
        nb_classes, nb_features = map(int, self.dist.scale.shape)

        # uniform priors
        priors = np.log(np.array([1. / nb_classes] * nb_classes))

        # Conditional probabilities log P(x|c)
        # (nb_samples, nb_classes, nb_features)
        print(6)
        all_log_probs = self.dist.log_prob(
            tf.reshape(
                tf.tile(X, [1, nb_classes]), [-1, nb_classes, nb_features]))
        # (nb_samples, nb_classes)
        cond_probs = tf.reduce_sum(all_log_probs, axis=2)

        # posterior log probability, log P(c) + log P(x|c)
        joint_likelihood = tf.add(priors, cond_probs)

        # normalize to get (log)-probabilities
        norm_factor = tf.reduce_logsumexp(
            joint_likelihood, axis=1, keepdims=True)
        log_prob = joint_likelihood - norm_factor
        print(7)
        # exp to get the actual probabilities
        return tf.exp(log_prob)

2


> # **Model**

In [ ]:
tf_nb = TFNaiveBayesClassifier()
tf_nb.fit(pd.DataFrame(X_train_matrix), y_train)

3
[    1     2     3 ... 13925 13926 13927]


In [ ]:
filename = 'finalized_model_tensor.sav'
pickle.dump(tf_nb, open(filename, 'wb'))

In [ ]:
text = ['мел']
seq = vect.transform(text)
pred = tf_nb.predict_proba(seq)
indices = np.argsort(pred[0])[-10:][::-1]
descr = [x[int(x)] for x in indices]
code = [y[int(x)] for x in indices]
output=[x for x in zip(descr, code)]
print(output)